In [3]:
%pip install opencv-python
%pip install mediapipe
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Start video capture
cap = cv2.VideoCapture(0)

def count_fingers(landmarks):
    finger_tips = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky
    finger_count = 0
    
    # Thumb check (using x-coordinate)
    if landmarks[4].x < landmarks[3].x:
        finger_count += 1
    
    # Other fingers (using y-coordinates)
    for tip in finger_tips[1:]:
        if landmarks[tip].y < landmarks[tip-2].y:
            finger_count += 1
            
    return finger_count

def recognize_gesture(landmarks):
    finger_count = count_fingers(landmarks)
    
    # Get key landmark positions
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    wrist = landmarks[0]
    
    # Yes: Thumb up (thumb extended, other fingers folded)
    if (thumb_tip.x < landmarks[3].x and 
        index_tip.y > landmarks[6].y and 
        middle_tip.y > landmarks[10].y):
        return "Yes"
    
    # No: Peace sign (2 fingers - index and middle)
    elif (finger_count == 2 and 
          index_tip.y < landmarks[6].y and 
          middle_tip.y < landmarks[10].y and 
          landmarks[16].y > landmarks[14].y):  # Ring finger down
        return "No"
    
    # Thanks: Open hand (5 fingers)
    elif finger_count == 5:
        return "Thanks"
    
    # Sorry: Fist with thumb touching fingers
    elif (finger_count == 0 and 
          abs(thumb_tip.x - index_tip.x) < 0.05 and 
          thumb_tip.y < landmarks[5].y):  # Thumb near fingers
        return "Sorry"
    
    # Default cases
    elif finger_count == 0:
        return "Closed Fist"
    else:
        return f"{finger_count} Fingers"

while True:
    success, frame = cap.read()
    if not success:
        break
        
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Recognize gesture
            gesture = recognize_gesture(hand_landmarks.landmark)
            
            # Display gesture
            cv2.putText(frame, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                       1, (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.imshow("Hand Gesture Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()